In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# save_path = '../results_final_Dec22/'
save_path = '../results_final_Jan15/'
signed_score_T_df = pd.read_csv(save_path + 'granger_hvg_T_signed_score.csv', index_col=0)
signed_score_A_df = pd.read_csv(save_path + 'granger_hvg_A_signed_score.csv', index_col=0)

In [18]:
# Convert to pairs with values for T
pairs = [(idx, col, val) for idx in signed_score_T_df.index for col, val in signed_score_T_df.abs().loc[idx].items()]

# Sort by absolute value
pairs.sort(key=lambda x: x[2], reverse=True)

# Get top pairs until we have 100 unique indices
seen_indices = set()
top_pairs = []
for idx, col, val in pairs:
    if idx not in seen_indices:
        seen_indices.add(idx)
        top_pairs.append((idx, col, val))
    if len(seen_indices) == 100:
        break

# Convert result to DataFrame for easier viewing
result = pd.DataFrame(top_pairs, columns=['Index', 'Column', 'Value'])
top_100_T = result['Index'].to_list()

# Now do the same for A
pairs = [(idx, col, val) for idx in signed_score_A_df.index for col, val in signed_score_A_df.abs().loc[idx].items()]

pairs.sort(key=lambda x: x[2], reverse=True)

seen_indices = set()
top_pairs = []
for idx, col, val in pairs:
    if idx not in seen_indices:
        seen_indices.add(idx)
        top_pairs.append((idx, col, val))
    if len(seen_indices) == 100:
        break

result_A = pd.DataFrame(top_pairs, columns=['Index', 'Column', 'Value'])
top_100_A = result_A['Index'].to_list()

In [19]:
# Find intersection between top 100 genes from T and A
intersection = set(top_100_T) & set(top_100_A)
print(f"Number of genes in intersection: {len(intersection)}")

Number of genes in intersection: 75


In [20]:
unique_T = set(top_100_T) - intersection
unique_A = set(top_100_A) - intersection
intersection_list = list(intersection)
unique_T_list = list(unique_T)
unique_A_list = list(unique_A)



In [25]:
# Create a dictionary of dataframes for each list
data_dict = {
    'Unique_T': pd.DataFrame(unique_T_list, columns=['Gene']),
    'Unique_A': pd.DataFrame(unique_A_list, columns=['Gene']), 
    'Intersection': pd.DataFrame(intersection_list, columns=['Gene']),
    'Top_100_T': pd.DataFrame(top_100_T, columns=['Gene']),
    'Top_100_A': pd.DataFrame(top_100_A, columns=['Gene'])
}

# Save to Excel with multiple sheets
with pd.ExcelWriter(save_path + 'granger_causality_rankings_hvg.xlsx') as writer:
    for sheet_name, df in data_dict.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)